In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4
import os

# Functions: get the correct grid cell in netcdf file and then grab the rainfall data

In [2]:
def get_coordinates_of_radar_gridcell(latitude_selection, longitude_selection, netcdf_file):
    
    #latitude and longitude selected by user (any lat, lon within the boundaries of radar file will work)
    lat_selection = latitude_selection
    lon_selection = longitude_selection
    
    #net cdf file read from the start
    ds = xr.open_dataset(netcdf_file)
    
    #create a list of all longitude and latitude coords from netcdf file
    longitudes = ds.x0.values.tolist()
    latitudes = ds.y0.values.tolist()

    #based on the user longitude selection, find the closest radar grid cell in netcdf file
    for i in range(0, len(longitudes)):
        if lon_selection == longitudes[i]:
            perf_lon = longitudes[i]   
        if i != len(longitudes) - 1:
            if longitudes[i + 1] > lon_selection and longitudes[i] < lon_selection:
                if abs(longitudes[i + 1] - lon_selection) > abs(longitudes[i] - lon_selection):
                    perf_lon = longitudes[i]
                else:
                    perf_lon = longitudes[i + 1]

    #gbased on the user latitude selection, find the closest radar grid cell in netcdf file
    for i in range(0, len(latitudes)):
        if lat_selection == latitudes[i]:
            perf_lat = latitudes[i]   
        if i != len(latitudes) - 1:
            if latitudes[i + 1] > lat_selection and latitudes[i] < lat_selection:
                if abs(latitudes[i + 1] - lat_selection) > abs(latitudes[i] - lat_selection):
                    perf_lat = latitudes[i]
                else:
                    perf_lat = latitudes[i + 1]
   
    #create a dictionary of the coordinates of closest grid cell to specified lat, long cords of user
    perfect_coordinates = {'latitude' : perf_lat, 'longitude' : perf_lon}
    
    return perfect_coordinates





def get_radar_rainfall(latitude_selection, longitude_selection, netcdf_file):
    #get the perfect coordinates of the grid cell based on the selection provided
    grid_cell_coords = get_coordinates_of_radar_gridcell(latitude_selection, longitude_selection, netcdf_file)

    #read the net cdf file in xarray : 
    xr_nc_file = xr.open_dataset(netcdf_file)

    #what is the longitude degree distance between each grid point?
    lon_dist = xr_nc_file.x0.values[0] - xr_nc_file.x0.values[1]
    #what is the latitude degree distance between each grid point?
    lat_dist = xr_nc_file.y0.values[0] - xr_nc_file.y0.values[1]
    #what size lat x lon box for averaging? (11x11). Create the box for getting the average rainfall
    lon_one = grid_cell_coords['longitude'] + lon_dist*5
    lon_two = grid_cell_coords['longitude'] - lon_dist*5
    lat_one = grid_cell_coords['latitude'] + lat_dist*5
    lat_two = grid_cell_coords['latitude'] - lat_dist*5

    #average rainfall from the box^
    avg_rf=xr_nc_file.sel(y0=slice(lat_one,lat_two),x0=slice(lon_one,lon_two)).mean(('y0','x0'), skipna=True)

    #REFORMAT THE DATA INTO A DATAFRAME
    rf_values = avg_rf.radarRF.values.tolist() #extract rainfall values into a list
    time_list = pd.to_datetime(avg_rf.time.values.tolist()) #extract time values into a list
    # create a dataframe of the time, rainfall, and coordinates
    df = pd.DataFrame()
    df['datetime'] = time_list
    df['rain_mm'] = rf_values
    df['lat'] = grid_cell_coords['latitude']
    df['lon'] = grid_cell_coords['longitude']

    #shift time to Hawaii time from UTC time
    df = df.set_index('datetime')
    df = df.shift(-10, freq='H') 
    df = df.reset_index()
    
    return df

# Define coordinate selection

In [3]:
#RADAR GAGE Hanalei WEST :  -159.4866399, 22.1662262
#RADAR GAGE Hanalei EAST : -159.4364186, 22.1504424
longitude_selection = -159.4364186
latitude_selection = 22.1504424

# Run through all of the radar files!

In [4]:
#get the correct order of files to download....
file_order={}
for file in os.listdir(os.path.join("..", "radar_files")): 
    number = int(file.split('H')[0])
    file_order[number] = file
sorted_files = dict(sorted(file_order.items()))

#collect all of the radar data
monthly_rain_gage_data = []
for file in range(1, len(sorted_files)+1):
    netcdf_file = os.path.join("..", "radar_files", sorted_files[file])
    rain_gage = get_radar_rainfall(latitude_selection, longitude_selection, netcdf_file)
    monthly_rain_gage_data.append(rain_gage)
    print(str(file), end = ' ')
all_rain_gage_data = pd.concat(monthly_rain_gage_data)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 

# Save to a csv file


In [5]:
#all_rain_gage_data.to_csv('HanaleiEast_radar.csv')